In [1]:
import os
import cv2
import numpy as np
from utils import get_face_landmarks


In [8]:
import pickle

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load data from the text file
data_file = "data.txt"
data = np.loadtxt(data_file)

# Split data into features (X) and labels (y)
X = data[:, :-1]  # Features are all columns except the last one
y = data[:, -1]   # Labels are the last column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier()

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = rf_classifier.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print(confusion_matrix(y_test, y_pred))

with open('./model', 'wb') as f:
    pickle.dump(rf_classifier, f)

Accuracy: 82.50%
[[ 7  0  0  3]
 [ 0 10  0  0]
 [ 1  1  8  0]
 [ 1  0  1  8]]


In [2]:
import os
import cv2
import numpy as np
from utils import get_face_landmarks

data_dir = '/Users/kevinguzmanhuamani/Desktop/faces'
output = []

# Iterar sobre las carpetas dentro de data_dir
for emotion_indx, emotion in enumerate(sorted(os.listdir(data_dir))):
    # Ignorar archivos ocultos como .DS_Store
    if emotion.startswith('.'):
        continue

    emotion_path = os.path.join(data_dir, emotion)
    # Verificar que sea un directorio
    if not os.path.isdir(emotion_path):
        continue

    # Iterar sobre las imágenes dentro de la carpeta de la emoción
    for image_file in os.listdir(emotion_path):
        # Ignorar archivos ocultos
        if image_file.startswith('.'):
            continue

        image_path = os.path.join(emotion_path, image_file)
        # Leer la imagen
        image = cv2.imread(image_path)
        if image is None:
            print(f"Advertencia: No se pudo leer la imagen {image_path}.")
            continue

        # Obtener los puntos de referencia faciales
        face_landmarks = get_face_landmarks(image)
        if len(face_landmarks) == 1404:
            face_landmarks.append(int(emotion_indx))
            output.append(face_landmarks)
        else:
            print(f"Advertencia: Puntos de referencia incompletos en {image_path}.")

# Guardar los datos en un archivo de texto
np.savetxt('data.txt', np.asarray(output))


I0000 00:00:1731106141.526049 8806738 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1731106141.534106 8806885 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731106141.536838 8806883 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1731106141.538940 8806882 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
/opt/anaconda3/lib/python3.12/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.war

In [9]:
import pickle

import cv2

from utils import get_face_landmarks


emotions = ['HAPPY', 'SAD', 'SURPRISED']

with open('./model', 'rb') as f:
    model = pickle.load(f)

cap = cv2.VideoCapture(2)

ret, frame = cap.read()

while ret:
    ret, frame = cap.read()

    face_landmarks = get_face_landmarks(frame, draw=True, static_image_mode=False)

    output = model.predict([face_landmarks])

    cv2.putText(frame,
                emotions[int(output[0])],
               (10, frame.shape[0] - 1),
               cv2.FONT_HERSHEY_SIMPLEX,
               3,
               (0, 255, 0),
               5)

    cv2.imshow('frame', frame)

    cv2.waitKey(25)


cap.release()
cv2.destroyAllWindows()

[11/08 18:02:30.561466][info][8806738][Context.cpp:69] Context created with config: default config!
[11/08 18:02:30.561494][info][8806738][Context.cpp:74] Context work_dir=/Users/kevinguzmanhuamani/Desktop/computer_vision/Emotion-Recognition
[11/08 18:02:30.561496][info][8806738][Context.cpp:77] 	- SDK version: 1.9.4
[11/08 18:02:30.561497][info][8806738][Context.cpp:78] 	- SDK stage version: main
[11/08 18:02:30.561499][info][8806738][Context.cpp:82] get config EnumerateNetDevice:false
[11/08 18:02:30.561502][info][8806738][MacPal.cpp:36] createObPal: create MacPal!
[11/08 18:02:30.561906][info][8806738][MacPal.cpp:104] Create PollingDeviceWatcher!
[11/08 18:02:30.561915][info][8806738][DeviceManager.cpp:15] Current found device(s): (0)
[11/08 18:02:30.561919][info][8806738][Pipeline.cpp:15] Try to create pipeline with default device.
[11/08 18:02:30.561920][warning][8806738][ObException.cpp:5] No device found, fail to create pipeline!
[11/08 18:02:30.588219][info][8806738][Context.cp

2024-11-08 18:02:30.558 python[75993:8806738] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
OpenCV: out device of bound (0-1): 2
OpenCV: camera failed to properly initialize!
[ WARN:0@813.484] global cap.cpp:323 open VIDEOIO(OBSENSOR): raised unknown C++ exception!


